##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizaje federado para clasificación de imágenes

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

**NOTA**: Esta colaboración ha sido verificada para trabajar con la [versión de lanzamiento más reciente](https://github.com/tensorflow/federated#compatibility) del paquete pip `tensorflow_federated`, pero el proyecto federado de TensorFlow aún se encuentra en una etapa de desarrollo previa al lanzamiento. Por lo tanto, es probable que no funcione en `main`.

En este tutorial usamos el ejemplo de entrenamiento clásico MNIST para presentar la capa de API de aprendizaje federado (FL, por sus siglas en inglés) de TFF, `tff.learning`; un conjunto de interfaces que se puede utilizar para realizar distintos tipos de tareas de aprendizaje federado, como un entrenamiento federado, con respecto a los modelos implementados por TensorFlow provistos por usuarios.

Este tutorial y la API de aprendizaje federado se diseñaron principalmente para usuarios que deseen conectar sus propios modelos de TensorFlow a TFF, tratando a este último principalmente como una caja negra. Para comprender mejor TFF y cómo implementar sus propios algoritmos de aprendizaje federado, consulte los tutoriales sobre la API FC Core: [algoritmos federados personalizados, parte 1](custom_federated_algorithms_1.ipynb) y [parte 2](custom_federated_algorithms_2.ipynb).

Para obtener más información sobre `tff.learning`, continúe con el tutorial [Aprendizaje federado para generación de textos](federated_learning_for_text_generation.ipynb), que además de cubrir modelos recurrentes, también demuestra cómo cargar un modelo Keras serializado previamente entrenado para refinarlo con aprendizaje federado combinado con evaluación usando Keras.

## Antes de empezar

Antes de empezar, ejecute lo que se encuentra a continuación, para asegurarse de que el entorno esté preparado correctamente. Si no ve un mensaje de inicio, consulte la guía de [instalación](../install.md) para obtener más instrucciones. 

In [ ]:
#@test {"skip": true}

!pip install --quiet --upgrade tensorflow-federated

In [ ]:
%load_ext tensorboard

Fetching TensorBoard MPM version 'live'... done.


In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Preparación de los datos de entrada

Empecemos con los datos. Para poner en práctica el aprendizaje federado es necesario contar con un conjunto de datos federados; es decir, una colección de datos de múltiples usuarios. Los datos federados normalmente son no [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables), lo que presenta un grupo de problemas particulares.

Para facilitar la experimentación, sembramos el repositorio de TFF con algunos conjuntos de datos, incluida una versión federada de MNIST que contiene una versión [conjunto de datos original de NIST](https://www.nist.gov/srd/nist-special-database-19) que ha sido reprocesado usando [Leaf](https://github.com/TalwalkarLab/leaf) para que los datos sean codificados por el escritor original de los dígitos. Dado que cada escritor tiene un estilo único, este conjunto de datos muestra el tipo de comportamiento no i.i.d. que se espera de los conjuntos de datos federados.

Podemos cargarlo de la siguiente manera.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Los conjuntos de datos que devuelve `load_data()` son instancias de `tff.simulation.ClientData`, una interfaz que le permite enumerar el conjunto de usuarios, construir un `tf.data.Dataset` que representa los datos de un usuario en particular y consultar la estructura de elementos individuales. A continuación, se explica cómo puede utilizar esta interfaz para explorar el contenido del conjunto de datos. Tenga en cuenta que, si bien esta interfaz le permite iterar sobre los identificadores de los clientes, esto es solo una característica de los datos de simulación. Como verá en breve, el marco de aprendizaje federado no utiliza identidades de clientes; su único propósito es permitirle seleccionar subconjuntos de datos para simulaciones.

In [ ]:
len(emnist_train.client_ids)

3383

In [ ]:
emnist_train.element_type_structure

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])

In [ ]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

example_element = next(iter(example_dataset))

example_element['label'].numpy()

1

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

### Exploración de la heterogeneidad en datos federados

Los datos federados normalmente no son [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables) y los usuarios suelen tener diferentes distribuciones de datos según los patrones de uso. Algunos clientes pueden tener menos ejemplos de entrenamiento en el dispositivo, ya que sufren escasez de datos a nivel local, mientras que otros clientes tendrán ejemplos de entrenamiento más que suficientes. Exploremos este concepto de heterogeneidad de datos típico de un sistema federado con los datos de EMNIST que tenemos disponibles. Es importante tener en cuenta que este análisis profundo de los datos de un cliente solo está disponible para nosotros porque se trata de un entorno de simulación donde todos los datos están disponibles para nosotros localmente. En un entorno federado de producción real, no sería posible inspeccionar los datos de un solo cliente.

Primero, tomemos una muestra de los datos de un cliente para tener una idea de los ejemplos en un dispositivo simulado. Debido a que el conjunto de datos que estamos usando fue codificado por un escritor único, los datos de un cliente representan la escritura a mano de una persona para una muestra de los dígitos del 0 al 9, simulando el "patrón de uso" único de un usuario.

In [ ]:
## Example MNIST digits for one client
figure = plt.figure(figsize=(20, 4))
j = 0

for example in example_dataset.take(40):
  plt.subplot(4, 10, j+1)
  plt.imshow(example['pixels'].numpy(), cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

Ahora visualicemos la cantidad de ejemplos en cada cliente para cada etiqueta de dígito MNIST. En el entorno federado, la cantidad de ejemplos en cada cliente puede variar bastante, según el comportamiento del usuario.

In [ ]:
# Number of examples per layer for a sample of clients
f = plt.figure(figsize=(12, 7))
f.suptitle('Label Counts for a Sample of Clients')
for i in range(6):
  client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[i])
  plot_data = collections.defaultdict(list)
  for example in client_dataset:
    # Append counts individually per label to make plots
    # more colorful instead of one color per plot.
    label = example['label'].numpy()
    plot_data[label].append(label)
  plt.subplot(2, 3, i+1)
  plt.title('Client {}'.format(i))
  for j in range(10):
    plt.hist(
        plot_data[j],
        density=False,
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

Ahora visualicemos la imagen media por cliente para cada etiqueta MNIST. Este código producirá la media de cada valor de píxel para todos los ejemplos del usuario para una etiqueta. Veremos que la imagen media de un cliente para un dígito se verá diferente a la imagen media de otro cliente para el mismo dígito, debido al estilo de escritura único de cada persona. Podemos reflexionar sobre cómo cada ronda de entrenamiento local empujará el modelo en una dirección diferente en cada cliente, a medida que aprendemos de los datos únicos de ese usuario en esa ronda local. Más adelante en el tutorial veremos cómo podemos tomar cada actualización del modelo de todos los clientes y agregarlas en nuestro nuevo modelo global, que ha aprendido de los datos únicos de cada uno de nuestros clientes.

In [ ]:
# Each client has different mean images, meaning each client will be nudging
# the model in their own directions locally.

for i in range(5):
  client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[i])
  plot_data = collections.defaultdict(list)
  for example in client_dataset:
    plot_data[example['label'].numpy()].append(example['pixels'].numpy())
  f = plt.figure(i, figsize=(12, 5))
  f.suptitle("Client #{}'s Mean Image Per Label".format(i))
  for j in range(10):
    mean_img = np.mean(plot_data[j], 0)
    plt.subplot(2, 5, j+1)
    plt.imshow(mean_img.reshape((28, 28)))
    plt.axis('off')

Los datos del usuario pueden ser ruidosos y estar etiquetados de manera poco confiable. Por ejemplo, al observar los datos del Cliente n.º 2 anteriores, podemos ver que para la etiqueta 2, es posible que haya habido algunos ejemplos mal etiquetados que crearon una imagen media más ruidosa.

### Preprocesamiento de los datos de entrada

Dado que los datos ya son `tf.data.Dataset`, el preprocesamiento se puede lograr mediante transformaciones de conjuntos de datos. Aquí, aplanamos las imágenes de `28x28` en arreglos de `784` elementos, mezclamos los ejemplos individuales, los organizamos en lotes y cambiamos el nombre de las características de `pixels` y `label` a `x` para usarlas con Keras. También incluimos una `repeat` del conjunto de datos para ejecutar varias épocas.

In [ ]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

Verifiquemos si funcionó.

In [ ]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

OrderedDict([('x', array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)), ('y', array([[2],
       [1],
       [5],
       [7],
       [1],
       [7],
       [7],
       [1],
       [4],
       [7],
       [4],
       [2],
       [2],
       [5],
       [4],
       [1],
       [1],
       [0],
       [0],
       [9]], dtype=int32))])

Ya tenemos casi todos los bloques de creación necesarios para construir conjuntos de datos federados.

Una de las formas de cargar datos federados a TFF en una simulación es simplemente como una lista de Python, en la que cada elemento de la lista contiene los datos de un usuario individual, ya sea como una lista o como un `tf.data.Dataset`. Como ya tenemos una interfaz que ofrece esto último, usémosla.

Aquí, presentamos una función ayudante simple que construirá una lista de conjuntos de datos (a partir de un conjunto dado de usuarios) como entrada a una ronda de entrenamiento o evaluación.

In [ ]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

Ahora, ¿cómo elegimos a los clientes?

En un escenario de entrenamiento federado típico, nos encontramos con una población potencialmente muy grande de dispositivos de usuario, de los cuales solo una fracción puede estar disponible para el entrenamiento en un momento determinado. Este es el caso, por ejemplo, cuando los dispositivos cliente son teléfonos móviles que participan en el entrenamiento solo cuando están conectados a una fuente de energía, fuera de una red medida y, en general, inactivos.

Por supuesto, estamos en un entorno de simulación y todos los datos están disponibles a nivel local. Por lo general, cuando ejecutamos simulaciones, simplemente tomamos muestras de un subconjunto aleatorio de los clientes que participarán en cada ronda de entrenamiento, que generalmente son diferentes en cada ronda.

Dicho esto, como podrá comprobar si estudia el documento sobre el algoritmo de [promediado federado](https://arxiv.org/abs/1602.05629), lograr la convergencia en un sistema con subconjuntos de clientes seleccionados aleatoriamente en cada ronda puede llevar un tiempo, y no sería práctico tener que ejecutar cientos de rondas en este tutorial interactivo.

Lo que haremos en su lugar es tomar muestras del conjunto de clientes una vez y reutilizar el mismo conjunto en rondas para acelerar la convergencia (sobreajustando intencionalmente los datos de estos pocos usuarios). Como ejercicio, el lector puede modificar este tutorial para simular la toma de muestras aleatorias; es una tarea bastante fácil (una vez que lo haga, tenga en cuenta que lograr que el modelo converja podría llevar un tiempo).

In [ ]:
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(emnist_train, sample_clients)

print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

Number of client datasets: 10
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>


## Creación de un modelo con Keras

Si usa Keras, probablemente ya tenga el código que construye un modelo Keras. A continuación, mostramos un ejemplo de un modelo simple que bastará para nuestro propósito.

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

**Nota:** Aún no compilamos el modelo. La pérdida, las métricas y los optimizadores se presentan más adelante.

Para utilizar cualquier modelo con TFF, es necesario incluirlo en una instancia de la interfaz `tff.learning.models.VariableModel`, que expone métodos para marcar el paso directo del modelo, las propiedades de los metadatos, etc., de manera similar a Keras, pero también introduce elementos adicionales, como formas de controlar el proceso de cálculo de métricas federadas. No nos preocupemos por esto por ahora; Si tiene un modelo de Keras como el que acabamos de definir anteriormente, puede hacer que TFF lo envuelva por usted al invocar `tff.learning.models.from_keras_model`, pasando el modelo y un lote de datos de muestra como argumentos, como se muestra a continuación.

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Entrenamiento del modelo con datos federados

Ahora que tenemos un modelo envuelto como `tff.learning.models.VariableModel` para usar con TFF, podemos dejar que TFF construya un algoritmo de promediado federado invocando la función ayudante `tff.learning.algorithms.build_weighted_fed_avg`, como se muestra a continuación.

Tenga en cuenta que el argumento debe ser un constructor (como `model_fn` arriba), no una instancia ya construida, para que la construcción de su modelo pueda ocurrir en un contexto controlado por TFF (si le interesa conocer las razones de esto, le recomendamos que lea el tutorial de seguimiento sobre [algoritmos personalizados](custom_federated_algorithms_1.ipynb)).

Una observación importante sobre el algoritmo de promediado federado que se muestra a continuación es que hay **2** optimizadores: *client_optimizer* y *server_optimizer*. El optimizador *client_optimizer* solo se usa para calcular actualizaciones del modelo local en cada cliente. El optimizador *server_optimizer* aplica la actualización promediada al modelo global en el servidor. En concreto, esto significa que quizá deba elegir un optimizador y una tasa de aprendizaje distintos de los que usó para entrenar el modelo en un conjunto de datos i.i.d. estándar. Recomendamos comenzar con un SGD normal, posiblemente con una tasa de aprendizaje menor de la habitual. La tasa de aprendizaje que usamos no se ajustó cuidadosamente, siéntase libre de experimentar.

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

¿Qué acaba de suceder? TFF ha construido un par de *cálculos federados* y los empaquetó en un `tff.templates.IterativeProcess` en el que estos cálculos están disponibles como un par de propiedades `initialize` y `next`.

En pocas palabras, *los cálculos federados* son programas en el lenguaje interno de TFF que pueden expresar varios algoritmos federados (consulte más información al respecto en el tutorial de [algoritmos personalizados](custom_federated_algorithms_1.ipynb)). En este caso, los dos cálculos generados y empaquetados en `iterative_process` implementan el [promedio federado](https://arxiv.org/abs/1602.05629).

Uno de los objetivos de TFF es definir los cálculos de tal forma que puedan ejecutarse en entornos reales de aprendizaje federado, pero actualmente solo se implementa el tiempo de ejecución de simulación de ejecución local. Para ejecutar un cálculo en un simulador, simplemente invóquelo como una función de Python. Este entorno interpretado de forma predeterminada no está diseñado para ofrecer un gran rendimiento, pero será suficiente para este tutorial; esperamos proporcionar tiempos de ejecución de simulación de mayor rendimiento para facilitar la investigación a mayor escala en versiones futuras.

Comencemos con el cálculo `initialize`. Como pasa con todos los cálculos federados, se puede considerar como una función. El cálculo no toma argumentos y devuelve un resultado: la representación del estado del proceso de promediado federado en el servidor. Aunque no queremos entrar en los detalles de TFF, quizá resulte útil ver cómo es este estado. Puede verlo de la siguiente manera.

In [ ]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[784,10],
      float32[10]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[784,10],
    float32[10]
  >
>@SERVER)


Si bien la firma de tipo anterior puede parecer un poco críptica al principio, puede reconocer que el estado del servidor consta de `global_model_weights` (los parámetros del modelo inicial para MNIST que se distribuirán a todos los dispositivos), algunos parámetros vacíos (como `distributor`, que controla la comunicación de servidor a cliente) y un componente `finalizer`. Este último controla la lógica que utiliza el servidor para actualizar su modelo al final de una ronda y contiene un número entero que representa cuántas rondas de FedAvg se ejecutaron.

Invoquemos el cálculo `initialize` para construir el estado del servidor.

In [ ]:
train_state = training_process.initialize()

El segundo par de cálculos federados, `next`, representa a una única ronda de promediado federado, que consiste en enviar el estado del servidor (incluidos los parámetros del modelo) a los clientes, el entrenamiento en el dispositivo sobre sus datos locales, la recopilación y promediado de las actualizaciones del modelo, y la producción de un nuevo modelo que se actualiza en el servidor.

A nivel conceptual, se puede pensar que `next` tiene una firma de tipo funcional con el siguiente aspecto.

```
SERVER_STATE, FEDERATED_DATA -&gt; SERVER_STATE, TRAINING_METRICS
```

En particular, uno no debería pensar en `next()` como una función que se ejecuta en un servidor, sino más bien como una representación funcional declarativa de todo el cálculo descentralizado: el servidor proporciona algunas de las entradas (`SERVER_STATE`), pero cada dispositivo participante aporta su propio conjunto de datos local.

Ejecutemos una única ronda de entrenamiento y observemos los resultados. Podemos usar los datos federados que ya generamos anteriormente como muestra de usuarios.

In [ ]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
train_metrics = result.metrics
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.12345679), ('loss', 3.1193733), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


Ejecutemos algunas rondas más. Tal como lo señalamos antes, normalmente en esta instancia, elegiríamos un subconjunto de los datos de simulación a partir de una muestra de usuarios seleccionada de forma aleatoria para cada ronda, a fin de simular una implementación realista en la cual los usuarios entran y salen continuamente. Pero en estas notas interactivas, con propósito demostrativo, nos limitaremos a reutilizar los mismos usuarios, para que el sistema converja rápidamente.

In [ ]:
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.14012346), ('loss', 2.9851403), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.1590535), ('loss', 2.8617127), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.17860082), ('loss', 2.7401376), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('fin

La pérdida de entrenamiento disminuye después de cada ronda de entrenamiento federado. Es señal de que el modelo está convergiendo. No obstante, hay algunas salvedades importantes relacionadas con estas métricas de entrenamiento. Si desea conocerlas, consulte la sección *Evaluación* que se trata más adelante en este tutorial.

## Visualización de las métricas del modelo en TensorBoard

A continuación, observemos las métricas de estos cálculos federados en TensorBoard.

Comencemos por crear un directorio y el escritor de resúmenes correspondiente en el que se redactarán las métricas.


In [ ]:
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
try:
  tf.io.gfile.rmtree(logdir)  # delete any previous results
except tf.errors.NotFoundError as e:
  pass # Ignore if the directory didn't previously exist.
summary_writer = tf.summary.create_file_writer(logdir)
train_state = training_process.initialize()

Tracemos las métricas escalares relevantes con el mismo escritor de resúmenes.

In [ ]:
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    result = training_process.next(train_state, federated_train_data)
    train_state = result.state
    train_metrics = result.metrics
    for name, value in train_metrics['client_work']['train'].items():
      tf.summary.scalar(name, value, step=round_num)

Iniciemos TensorBoard con el directorio de registros raíz que se especifica arriba. La carga de los datos puede demorar algunos segundos.

In [ ]:
#@test {"skip": true}
!ls {logdir}
%tensorboard --logdir {logdir} --port=0

In [ ]:
#@test {"skip": true}
# Uncomment and run this cell to clean your directory of old output for
# future graphs from this directory. We don't run it by default so that if 
# you do a "Runtime > Run all" you don't lose your results.

# !rm -R /tmp/logs/scalars/*

A fin de ver las métricas de evaluación del mismo modo, se puede crear una carpeta de evaluación por separado, como "logs/scalars/eval", para escribir en TensorBoard.

## Personalización de la implementación del modelo

Keras es la [API de modelo de alto nivel recomendada para TensorFlow](https://medium.com/tensorflow/standardizing-on-keras-guidance-on-high-level-apis-in-tensorflow-2-0-bad2b04c819a) y promovemos el uso de modelos Keras (a través de `tff.learning.models.from_keras_model`) en TFF siempre que sea posible.

No obstante, `tff.learning` ofrece una interfaz de modelo de nivel inferior, `tff.learning.models.VariableModel`, que expone la funcionalidad mínima necesaria para usar un modelo para el aprendizaje federado. La implementación directa de esta interfaz (que probablemente todavía use bloques de construcción como `tf.keras.layers`) permite alcanzar la máxima personalización sin modificar los aspectos internos de los algoritmos de aprendizaje federados.

Así que, empecemos todo de nuevo desde cero.

### Definición de variables del modelo, paso hacia adelante y métricas

El primer paso consiste en identificar las variables de TensorFlow con las que vamos a trabajar. Para que el siguiente código sea más legible, definamos una estructura de datos para representar el conjunto completo. Esto incluirá variables como `weights` y `bias` que entrenaremos, así como variables que contendrán varias estadísticas acumulativas y contadores que actualizaremos durante el entrenamiento, como `loss_sum`, `accuracy_sum` y `num_examples`.

In [ ]:
MnistVariables = collections.namedtuple(
    'MnistVariables', 'weights bias num_examples loss_sum accuracy_sum')

Este es un método que crea las variables. Para que resulte más sencillo, representamos todas las estadísticas como `tf.float32`, ya que eso eliminará la necesidad de realizar conversiones de tipos en una etapa posterior. Envolver inicializadores de variables como lambdas es un requisito impuesto por [las variables de recursos](https://www.tensorflow.org/api_docs/python/tf/enable_resource_variables).

In [ ]:
def create_mnist_variables():
  return MnistVariables(
      weights=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(784, 10)),
          name='weights',
          trainable=True),
      bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(10)),
          name='bias',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False))

Con las variables para los parámetros del modelo y las estadísticas acumuladas ya establecidas, ahora podemos definir el método de paso hacia adelante que calcula la pérdida, emite predicciones y actualiza las estadísticas acumuladas para un único lote de datos de entrada, de la siguiente manera.

In [ ]:
def predict_on_batch(variables, x):
  return tf.nn.softmax(tf.matmul(x, variables.weights) + variables.bias)

def mnist_forward_pass(variables, batch):
  y = predict_on_batch(variables, batch['x'])
  predictions = tf.cast(tf.argmax(y, 1), tf.int32)

  flat_labels = tf.reshape(batch['y'], [-1])
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 10) * tf.math.log(y), axis=[1]))
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))

  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  return loss, predictions

A continuación, volvemos a usar TensorFlow, esta vez para definir dos funciones que están relacionadas con métricas locales.

La primera función `get_local_unfinalized_metrics` devuelve los valores de métricas sin finalizar (además de las actualizaciones del modelo, que se gestionan automáticamente) que son elegibles para agregarse al servidor en un proceso de evaluación o aprendizaje federado.

In [ ]:
def get_local_unfinalized_metrics(variables):
  return collections.OrderedDict(
      num_examples=[variables.num_examples],
      loss=[variables.loss_sum, variables.num_examples],
      accuracy=[variables.accuracy_sum, variables.num_examples])

La segunda función `get_metric_finalizers` devuelve un `OrderedDict` de `tf.function`s con las mismas claves (es decir, nombres de métricas) que `get_local_unfinalized_metrics`. Cada `tf.function` toma los valores sin finalizar de la métrica y calcula la métrica finalizada.

In [ ]:
def get_metric_finalizers():
  return collections.OrderedDict(
      num_examples=tf.function(func=lambda x: x[0]),
      loss=tf.function(func=lambda x: x[0] / x[1]),
      accuracy=tf.function(func=lambda x: x[0] / x[1]))

La forma en que se agregan las métricas locales sin finalizar que devuelve `get_local_unfinalized_metrics` entre los clientes se especifica mediante el parámetro `metrics_aggregator` al definir los procesos de evaluación o aprendizaje federados. Por ejemplo, en la API [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) (que se muestra en la siguiente sección), el valor predeterminado para `metrics_aggregator` es [`tff.learning.metrics.sum_then_finalize`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/metrics/sum_then_finalize), que primero suma las métricas sin finalizar de `CLIENTS` y ​​luego aplica los finalizadores de métricas en `SERVER`.

### Construcción de una instancia de `tff.learning.models.VariableModel`

Una vez que hayamos implementado todo lo anterior, podremos construir una representación del modelo para usar con TFF que sea similar a la que se genera cuando permitimos que TFF incorpore un modelo de Keras.

In [ ]:
import collections
from collections.abc import Callable

class MnistModel(tff.learning.models.VariableModel):

  def __init__(self):
    self._variables = create_mnist_variables()

  @property
  def trainable_variables(self):
    return [self._variables.weights, self._variables.bias]

  @property
  def non_trainable_variables(self):
    return []

  @property
  def local_variables(self):
    return [
        self._variables.num_examples, self._variables.loss_sum,
        self._variables.accuracy_sum
    ]

  @property
  def input_spec(self):
    return collections.OrderedDict(
        x=tf.TensorSpec([None, 784], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int32))

  @tf.function
  def predict_on_batch(self, x, training=True):
    del training
    return predict_on_batch(self._variables, x)
    
  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = mnist_forward_pass(self._variables, batch)
    num_exmaples = tf.shape(batch['x'])[0]
    return tff.learning.models.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_exmaples)

  @tf.function
  def report_local_unfinalized_metrics(
      self) -> collections.OrderedDict[str, list[tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to unfinalized values."""
    return get_local_unfinalized_metrics(self._variables)

  def metric_finalizers(
      self) -> collections.OrderedDict[str, Callable[[list[tf.Tensor]], tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to finalizers."""
    return get_metric_finalizers()

  @tf.function
  def reset_metrics(self):
    """Resets metrics variables to initial value."""
    for var in self.local_variables:
      var.assign(tf.zeros_like(var))

Como puede ver, los métodos abstractos y las propiedades definidas por `tff.learning.models.VariableModel` corresponden a los fragmentos de código de la sección anterior que introdujeron las variables y definieron la pérdida y las estadísticas.

Vale la pena resaltar algunos puntos:

- Todos los estados que usará su modelo se deben capturar como variables de TensorFlow, ya que TFF no usa Python en tiempo de ejecución (recuerde que su código debe escribirse de manera que pueda implementarse en dispositivos móviles; consulte el tutorial de [algoritmos personalizados](custom_federated_algorithms_1.ipynb) para acceder a información más detallada sobre las razones).
- Su modelo debe describir qué forma de datos acepta (`input_spec`), ya que, en general, TFF es un entorno fuertemente tipado y tiende a determinar firmas de tipo para todos los componentes. Declarar el formato de la entrada de su modelo es fundamental.
- Si bien técnicamente no es necesario, recomendamos envolver toda la lógica de TensorFlow (paso hacia adelante, cálculos métricos, etc.) como `tf.function`s, ya que esto ayuda a garantizar que TensorFlow se pueda serializar y elimina la necesidad de contar con dependencias de control explícitas.


Con lo expuesto anteriormente es suficiente para la evaluación y los algoritmos como Federated SGD. Sin embargo, para el promediado federado, debemos especificar el método de entrenamiento local del modelo en cada lote. Al crear el algoritmo de promediado federado, especificaremos un optimizador local.

### Simulación del entrenamiento federado con el nuevo modelo

Una vez que haya hecho todo lo anterior, el resto del proceso se parece a lo que ya hemos visto: simplemente reemplace el constructor del modelo con el constructor de nuestra nueva clase de modelo y use los dos cálculos federados en el proceso iterativo que creó para recorrer rondas de entrenamiento.

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    MnistModel,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02))

In [ ]:
train_state = training_process.initialize()

In [ ]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 3.119374), ('accuracy', 0.12345679)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [ ]:
for round_num in range(2, 11):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.98514), ('accuracy', 0.14012346)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.8617127), ('accuracy', 0.1590535)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.740137), ('accuracy', 0.17860082)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  5, metrics=OrderedDict([('distributor', ()), ('client_work', 

Para ver estas métricas dentro de TensorBoard, consulte los pasos mencionados anteriormente en "Visualización de las métricas del modelo en TensorBoard".

## Evaluación

Hasta el momento, todos nuestros experimentos presentaron solo métricas de entrenamiento federadas: las métricas promedio de todos los lotes de datos entrenados en todos los clientes de la ronda. Esto plantea el clásico problema del sobreajuste, sobre todo porque, para simplificar, usamos el mismo conjunto de clientes en cada ronda, pero existe una noción adicional de sobreajuste en las métricas de entrenamiento específicas del algoritmo de promediado federado. Esto es más fácil de ver si imaginamos que cada cliente tiene un solo lote de datos y entrenamos en ese lote durante muchas iteraciones (épocas). En este caso, el modelo local rápidamente se ajustará exactamente a ese lote, por lo que la métrica de precisión local que promediamos se aproximará a 1,0. Así, estas métricas de entrenamiento pueden tomarse como una señal de que el entrenamiento está progresando, pero no mucho más.

Para evaluar los datos federados, puede construir otro *cálculo federado* diseñado precisamente para este propósito, que utilice la función `tff.learning.build_federated_evaluation` y pase el constructor de su modelo como argumento. Tenga en cuenta que, a diferencia del promediado federado, donde usamos `MnistTrainableModel`, basta con pasar `MnistModel`. La evaluación no realiza un descenso del gradiente y no es necesario construir optimizadores.

Para experimentación e investigación, cuando hay un conjunto de datos de prueba centralizado disponible, [Aprendizaje federado para clasificación de imágenes](federated_learning_for_text_generation.ipynb) demuestra otra opción de evaluación: tomar las ponderaciones entrenadas del aprendizaje federado, aplicarlas a un modelo Keras estándar y luego simplemente llamar `tf.keras.models.Model.evaluate()` en un conjunto de datos centralizado.

In [ ]:
evaluation_process = tff.learning.algorithms.build_fed_eval(MnistModel)

Puede inspeccionar la firma de tipo abstracto de la función de evaluación de la siguiente manera.

In [ ]:
print(evaluation_process.next.type_signature.formatted_representation())

(<
  state=<
    global_model_weights=<
      trainable=<
        float32[784,10],
        float32[10]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        num_examples=<
          float32
        >,
        loss=<
          float32,
          float32
        >,
        accuracy=<
          float32,
          float32
        >
      >
    >,
    aggregator=<
      value_sum_process=<>,
      weight_sum_process=<>
    >,
    finalizer=<>
  >@SERVER,
  client_data={<
    x=float32[?,784],
    y=int32[?,1]
  >*}@CLIENTS
> -> <
  state=<
    global_model_weights=<
      trainable=<
        float32[784,10],
        float32[10]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        num_examples=<
          float32
        >,
        loss=<
          float32,
          float32
        >,
        accuracy=<
          float32,
          float32
        >
      >
    >,
    aggregator=<
      value_

Tenga en cuenta que el proceso de evaluación es un objeto `tff.lenaring.templates.LearningProcess`. El objeto tiene un método `initialize` que creará el estado, pero al principio contendrá un modelo sin entrenar. Inserte las ponderaciones del estado de entrenamiento a evaluar con ayuda del método `set_model_weights`.

In [ ]:
evaluation_state = evaluation_process.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

Ahora que el estado de evaluación contiene las ponderaciones del modelo que se van a evaluar, podemos usar los conjuntos de datos de evaluación para calcular las métricas de evaluación si llamamos al método `next` en el proceso, como se hizo en el entrenamiento.

Esto volverá a devolver una instancia `tff.learning.templates.LearingProcessOutput`.

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_train_data)

Esto es lo que obtenemos. Como podemos observar, al parecer las cifras son ligeramente mejores que las que se informaron en la última ronda de entrenamiento anterior. Por convención, las métricas de entrenamiento reportadas por el proceso de entrenamiento iterativo generalmente reflejan el desempeño del modelo al inicio de la ronda de entrenamiento, por lo que las métricas de evaluación siempre estarán un paso adelante.

In [ ]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('num_examples', 4860.0), ('loss', 1.6654209), ('accuracy', 0.3621399)])), ('total_rounds_metrics', OrderedDict([('num_examples', 4860.0), ('loss', 1.6654209), ('accuracy', 0.3621399)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

Ahora compilemos una muestra de prueba de datos federados y volvamos a ejecutar la evaluación de los datos de prueba. Los datos provendrán de la misma muestra de usuarios, pero de un conjunto de datos retenidos distinto.

In [ ]:
federated_test_data = make_federated_data(emnist_test, sample_clients)

len(federated_test_data), federated_test_data[0]

(10,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>)

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [ ]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('num_examples', 580.0), ('loss', 1.7750846), ('accuracy', 0.33620688)])), ('total_rounds_metrics', OrderedDict([('num_examples', 580.0), ('loss', 1.7750846), ('accuracy', 0.33620688)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

De este modo, se concluye con el tutorial. Le aconsejamos que pruebe con distintos parámetros (p. ej., los tamaños de los lotes, la cantidad de usuarios, las épocas, las tasas de aprendizaje, etc.), para modificar el código que figura arriba a fin de simular el entrenamiento con muestras aleatorias de usuarios en cada ronda. También le recomendamos explorar los otros tutoriales que hemos desarrollado.